# Candy Crush in Python - Part 4 - Removing Matches and Filling in Blank Spaces

In this lesson we will write add some interactivity to our grid, when the user clicks the two cells beside eachother in the grid, we want to swap the tiles and to run all of our other functions to find and remove matches, as well as refill blank spaces left over.

# Code Recap

So far we have:
* Our game board and function to display the grid
* A function to swap two cells beside eachother
* A function to find all matches of a particular length on the board
* A function to remove matches
* A function to refill the empty spaces either with the cells from above, or random cells

In [1]:
import numpy as np
from gamegrid import GameGrid
from IPython.display import display


def show_board(board_array):
    grid = GameGrid()
    grid.data = board_array
    display(grid)
    

def swap_items(board, first, second):    
    dist = np.sum(np.abs(np.subtract(second, first)))
    
    is_beside = dist <= 1
    
    if(is_beside):
        tmp = board[first]
        board[first] = board[second]        
        board[second] = tmp 


def horiz_match_length(board, point):
    first = board[point]
    (r, c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1
    
    for i in range(c+1, col_count):
        if(board[r, i] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count


def vert_match_length(board, point):
    first = board[point]
    (r,c) = point
    (row_count, col_count) = board.shape
    same_tile_count = 1    
    
    for i in range(r+1, row_count):
        if(board[i,c] != first):
            return same_tile_count
        else:
            same_tile_count = same_tile_count + 1
            
    return same_tile_count


def find_matches(board, required_matches):
    horiz_matches = []
    vert_matches = []
    
    (row_count, col_count) = board.shape
    
    for r in range(row_count):
        for c in range(col_count):
            point = (r,c)
            
            horiz_len = horiz_match_length(board, point)
            if horiz_len >= required_matches:
                horiz_matches.append((r, c, horiz_len))  
            
            vert_len = vert_match_length(board, point)
            if vert_len >= required_matches:
                vert_matches.append((r, c, vert_len))
                
    return (horiz_matches, vert_matches)


def remove_matching_tiles(board, horiz_matches, vert_matches):
    for (r, c, length) in horiz_matches:
         for i in range(c, c+length):
                board[r, i] = -1
    
    for (r, c, length) in vert_matches:
        for i in range(r, r+length):
            board[i, c] = -1
            
            
def pull_down_cell(board, point, number_of_types):
    (r, c) = point
    for r_above in reversed(range(r)):    
        if(board[r_above, c] != -1):            
            board[r,c] = board[r_above,c]
            board[r_above,c] = -1
            break
    if(board[r,c] == -1):
        board[r,c] = np.random.randint(number_of_types)
        
        
def pull_down_cells(board, number_of_types):
    for r in reversed(range(board.shape[0])):
        for c in range(board.shape[1]):
            if(board[r,c] == -1):
                pull_down_cell(board, (r, c), number_of_types)

## Click Handling

The GameGrid widget has a function called on_click, that allows us to add a function to be called whenever the grid is clicked. Lets try it out:

In [3]:
interactive_grid = GameGrid()
interactive_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,1],
    [2,1,4,3,2],
    [4,4,1,2,3]
])

def interactive_on_click(grid, row, col):
    print('Clicked (' + str(row) + ', ' + str(col) + ')')

interactive_grid.on_click(interactive_on_click)
display(interactive_grid)

GameGrid()

When you click one of the cells in this grid, it prints out the cell that was clicked. We can make this a little nicer to read using a lambda expression - which is just a way of writing a function in a single line:

In [5]:
interactive_grid_lambda = GameGrid()
interactive_grid_lambda.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,1],
    [2,1,4,3,2],
    [4,4,1,2,3]
])

interactive_grid_lambda.on_click(lambda g, r, c: print('(Lambda) Clicked (' + str(r) + ', ' + str(c) + ')'))
display(interactive_grid_lambda)

GameGrid()

## Swapping on Click

When we click on one cell, then another, we want it to swap those cells - i.e. to call our `swap_items(...)` function.

### Toggle Select

The GameGrid we use to display our board has a `toggle_select(...)` function, which will show the cell as selected. The function takes a tuple with the row and column of the point we want to select.

In [6]:
select_grid = GameGrid()
select_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,1],
    [2,1,4,3,2],
    [4,4,1,2,3]
])

display(select_grid)

# Select
select_grid.toggle_select((1,3))

GameGrid()

### Selection Click Handler

We are now ready to create a handler that will select the cell we clicked on:

In [8]:
def handle_click(grid, point):       
    grid.toggle_select(point)
    
select_grid = GameGrid()
select_grid.on_click(lambda g, r, c: handle_click(g, (r, c)))
select_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,1],
    [2,1,4,3,2],
    [4,4,1,2,3]
])

display(select_grid)

GameGrid()

When we click cells in this grid, it toggles the selection.

### Handle Click with Swapping

We can change our `handle_click(...)` function to do our swapping, which will do the following:

* If nothing is selected yet, then select what was clicked and return
* If the point we clicked is the one we selected previously, then deselect it and return
* Otherwise we have two different points, so try to swap them
  * We should then deselect the first point that was clicked
  * We need to also update the grid

In [9]:
def handle_click(grid, point):   
    
    grid.toggle_select(point)
    
    # If we have zero or 1 points selected we dont need to do anymore
    if len(grid.selected) < 2:
        return    
    
    # lets get the current board
    board = grid.data
    last_point = grid.selected[0]
    
    # We have two different points, so we try to swap them
    swap_items(board, last_point, point)
    
    # Deselect both points
    grid.toggle_select(last_point)
    grid.toggle_select(point)
    
    # Update the grid
    grid.data = board

We can test this out:

In [11]:
swap_click_grid = GameGrid()
swap_click_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,4],
    [2,1,4,3,2],
    [4,4,1,2,3]
])

swap_click_grid.on_click(
    lambda g, r, c: handle_click(g, (r, c)))

display(swap_click_grid)

GameGrid()

## Finding matches after we swap

After we have swapped we want to check and remove any matches, so we will extend our function to do the following:
- After we swap, find all matches matches of length 3 or more
- If there are matches
  - Remove all matches
  - Fill in the blank cells
- If there arent any matches
  - Swap the two cells back  

In [12]:
def handle_click(grid, point):   
    
    # Select the Point
    grid.toggle_select(point)
    
    # If we only have 0 or 1 points selected 
    #  then return right away
    if len(grid.selected) < 2:
        return    
    
    # We have two points selected, get the
    #  board and the first selected point
    board = grid.data
    last_point = grid.selected[0]
   
    # Swap Items
    swap_items(board, last_point, point)
    
    # Deselect both points
    grid.toggle_select(last_point)
    grid.toggle_select(point)
    
    # Find all matches
    (h_matches, v_matches) = find_matches(board, 3)
    match_count = len(h_matches) + len(v_matches)
    
    if(match_count > 0):          
        # We found some matches so remove them and re-fill the board
        remove_matching_tiles(board, h_matches, v_matches)
        # The number of types is the same as the number of images on our grid
        number_of_types = len(grid.images)
        pull_down_cells(board, number_of_types)
    else:
        # We havent found any matches so swap back
        swap_items(board, last_point, point)        
    
    # Update the grid with the new board
    grid.data = board    

And we can test this now:

In [13]:
swap_click_grid = GameGrid()
swap_click_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,4],
    [2,1,4,3,2],
    [4,4,1,2,3]
])

swap_click_grid.on_click(
    lambda g, r, c: handle_click(g, (r, c)))

display(swap_click_grid)

GameGrid()

This is starting to get close to our finished game - but there are a few issues remaining:
1. After we remove matches and re-fill, the grid might have more matches, we need to remove all of these also
2. All of the steps happen at the same time, so its hard to see what is going on
3. We only check for matches of 3 - we might want to make our game a little harder by checking for matches of 4 or more

## Processing Matches Recursively

As we noticed above, we only check for matches once. After we remove matches and fill in the blank cells we might have more matches - so we need to keep checking for matches and removing until there are no more matches - so lets create a function to handle this.

In our function, we will:
1. Check for matches
2. Remove matches and fill in the blank cells
3. If there were matches, then call itself again
4. Return the number of matches we found

We need to get back the matches we found so that we can decide if we need to swap back or not. 

In [14]:
def process_matches(board, number_of_types):
    
    # Find all matches on the board
    (h_matches, v_matches) = find_matches(board, 3)
    # Remove the matches
    remove_matching_tiles(board, h_matches, v_matches)
    # Check if we gont any matches
    match_count = len(h_matches) + len(v_matches)    
    
    if match_count > 0:
        # We have matches so pull down cells
        pull_down_cells(board, number_of_types)
        # We might have more matches, so call process_matches(board)
        #  again to remove them
        count = process_matches(board, number_of_types)
        match_count = match_count + count
        
    return match_count

We need to also update our handle_click function to use this function:

In [15]:
def handle_click(grid, point):   
    
    grid.toggle_select(point)
    
    if len(grid.selected) < 2:
        return    
    
    board = grid.data
    last_point = grid.selected[0]
   
    swap_items(board, last_point, point)
    
    grid.toggle_select(last_point)
    grid.toggle_select(point)
    
    number_of_types = len(grid.images)
    # Process all matches and get the total number found
    match_count = process_matches(board, number_of_types)
    
    if(match_count == 0):
        # We havent found any matches so swap back
        swap_items(board, last_point, point)        
    
    
    grid.data = board    

We can test these new functions as follows:

In [16]:
find_all_grid = GameGrid()
find_all_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,4],
    [2,1,4,3,2],
    [4,4,1,2,3]
])

find_all_grid.on_click(
    lambda g, r, c: handle_click(g, (r, c)))

display(find_all_grid)

GameGrid()

## Updating the Grid at every Step

Next we want to be able to see what is going on - we want to see every step that happens on the grid. To do this we need to update the grid each time we make a change. This will update the grid for each change, but it will probably happen too fast for us to see, so we would like to slow things down a bit. We can use the `time.sleep(...)` function to do this for us.

Lets create a function to update the grid, and to pause for a little bit after each change. We will pause for 1/4 a second, which is enough time to show what is happening.

In [17]:
import time
def update_grid(grid, board):
    grid.data = board
    time.sleep(0.25)

In our process matches function, we dont really want to put details of the updating in this function, so we can just pass it a a function called refresh that will do the actual update.

In [18]:
def process_matches(board, number_of_types, refresh):
    
    (h_matches, v_matches) = find_matches(board, 3)
    remove_matching_tiles(board, h_matches, v_matches)
    match_count = len(h_matches) + len(v_matches)    
    
    if match_count > 0:        
        refresh()
        pull_down_cells(board, number_of_types)
        refresh()
        count = process_matches(board, number_of_types, refresh)
        match_count = match_count + count
        
    return match_count

No we update our `handle_click(...)` function:

In [19]:
def handle_click(grid, point):   
        
    grid.toggle_select(point)
    
    if len(grid.selected) < 2:
        return    
    
    board = grid.data
    # Create our refresh function that will update the grid
    #  and pause so we can see the update
    refresh = lambda: update_grid(grid, board)
    last_point = grid.selected[0]
   
    swap_items(board, last_point, point)
    # We have made a change so refresh the grid
    refresh()
    
    grid.toggle_select(last_point)
    grid.toggle_select(point)
    
    number_of_types = len(grid.images)
    # We pass the refresh function to process_matches
    #  so that it can update the grid every time it changes
    #  something
    match_count = process_matches(board, number_of_types, refresh)
    
    if(match_count == 0):
        # We havent found any matches so swap back
        swap_items(board, last_point, point)      
        refresh()  

And we test:

In [21]:
find_all_grid = GameGrid()
find_all_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,4],
    [2,1,4,3,2],
    [4,4,1,2,3]
])

find_all_grid.on_click(
    lambda g, r, c: handle_click(g, (r, c)))

display(find_all_grid)

GameGrid()

### Allowing different number of matches

The last thing we want to do is to allow a different number of matches.

We currently pass the number 3 to the find_matches(...) function, we can simply change this to a variable to allow us to specify any number. So lets update our process_matches(...) function to make this a variable.

In [22]:
# Add the required_matches parameter
def process_matches(board, number_of_types, required_matches, refresh):
    
    # Pass in the required_matches
    (h_matches, v_matches) = find_matches(board, required_matches)
    remove_matching_tiles(board, h_matches, v_matches)
    match_count = len(h_matches) + len(v_matches)    
    
    if match_count > 0:        
        refresh()
        pull_down_cells(board, number_of_types)
        refresh()
        # Pass the required_matches to the next process_matches
        count = process_matches(board, number_of_types, required_matches, refresh)
        match_count = match_count + count
        
    return match_count

We call process_matches(...) in handle_click(...), so we must update that too:

In [23]:
def handle_click(grid, point, required_matches):   
        
    grid.toggle_select(point)
    
    if len(grid.selected) < 2:
        return    
    
    board = grid.data
    refresh = lambda: update_grid(grid, board)
    last_point = grid.selected[0]
   
    swap_items(board, last_point, point)
    refresh()
    
    grid.toggle_select(last_point)
    grid.toggle_select(point)
    
    number_of_types = len(grid.images)
    # Process all matches and get the total number found
    match_count = process_matches(board, number_of_types, required_matches, refresh)
    
    if(match_count == 0):
        # We havent found any matches so swap back
        swap_items(board, last_point, point)      
        refresh()

Now lets test with a new grid:

In [24]:
find_all_grid = GameGrid()
find_all_grid.data = np.array([
    [1,2,1,4,1],
    [3,3,2,3,4],
    [2,1,3,3,2],
    [4,4,1,2,3]
])

required_matches = 4

find_all_grid.on_click(
    lambda g, r, c: handle_click(g, (r, c), required_matches))

display(find_all_grid)

GameGrid()

## Summary

In this lesson we learned how add interactivity to our grid. We can click select two cells on the grid, swap them and process any matches we find.

In the next section we will be [Putting Everything Together](Part6-PuttingItTogether.ipynb)